### Import and data loading

In [30]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.metrics.cluster import contingency_matrix
import numpy as np

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import DBSCAN

from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import scipy
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, chi2
from sklearn.model_selection import cross_val_predict, train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
pd.options.display.max_rows = 1000

In [148]:
#Import data
kilothonGames = pd.read_csv('KilothonGames.csv')
concepts = pd.read_csv('concepts.csv')
gamerules = pd.read_csv('gamerulesets.csv')
games = pd.read_csv('games.csv')
rulesetconcept = pd.read_csv('rulesetconcepts.csv')
rsc_uct = pd.read_csv('rulesetconceptsuct.csv')
rsc_ab = pd.read_csv('rulesetconceptsab.csv')

#### Reading extra data for the analysis

In [304]:
heurs = pd.read_csv('heuristics.csv')
abgames = pd.read_csv('ABGames.csv')
preds = pd.read_csv('predictions.csv')
finaltime = pd.read_csv('TimeOnly.csv')
heurandtime = pd.read_csv('TimeAndHeur.csv')
withheurs = pd.read_csv('resultsHeuristics.csv')

#### Methods for processing the outputs of a kilothon

In [279]:
def get_utilities(finalresults):
    combined = pd.merge(finalresults, preds, left_on='GAME NAME', right_on='Name')
    print(finalresults[' UTILITY P1'].mean())
    for i in ['', 0,1,2,3,4]:
        zeros = combined[combined['BestAI_pred'] == i]
        utility = zeros[' UTILITY P1'].mean()
        print(i, ' ', utility, " #", len(zeros))
        print(utility*len(zeros))
        
def get_winrate(finalresults):
    combined = pd.merge(finalresults, preds, left_on='GAME NAME', right_on='Name')
    for i in [0,1,2,3,4]:
        zeros = combined[combined['BestAI_pred'] == i]
        zeroswon = len(zeros[zeros[' UTILITY P1']==1.0])
        zeroslost = len(zeros[zeros[' UTILITY P1']==-1.0])
        if zeroslost != 0:
            zerosacc = zeroswon/(zeroswon+zeroslost)
            print(i, " ", zerosacc, " #", len(zeros))    

### Get infos of the different database

As discussed, we will only keep the following columns for our final datframe:

- Concepts: ID, LeafNode
- Gamerules: ID, GameID, Name, Type
- Games: ID, Name, MainRuleSet, LudiiRuleset
- Rulesetconcept: ID, RuleSetId, ConceptID, value 

In [40]:
concepts = concepts[~concepts['TaxonomyString'].str.startswith('6')]
concepts = concepts[concepts['LeafNode']!=0]

In [41]:
concepts = concepts.drop([ 'Unnamed: 0', 'Description', 'TypeId', 'DataTypeId', 'ComputationTypeId', 'TaxonomyString'], axis = 1);
gamerules = gamerules.drop(['Unnamed: 0', 'NativeName', 'Summary', 'Description', 'Rules', 'Reference', 'Origin', 'Author', 'Publisher', 'Date', 'OriginPoint', 'EvidenceRange', 'SelfContained', 'Notes', 'DisableWebApp', 'WishlistRuleset', 'Type'], axis = 1);
games = games.drop(['Unnamed: 0','NativeName', 'Description', 'Reference', 'Origin', 'DLPGame', 'PublicGame', 'knownAliases', 'Author', 'Publisher', 'ProprietaryGame', 'Credit', 'SeeAlso', 'BGGId', 'OriginPoint', 'EvidenceRange', 'WishlistGame', 'Notes', 'HelpUs', 'ForceRulesetInLud', 'DisableWebApp', 'ReconstructionDescriptionDone'], axis = 1);
rulesetconcept = rulesetconcept.drop(['Unnamed: 0'], axis = 1)
rsc_uct = rsc_uct.drop(['Unnamed: 0'], axis = 1)
rsc_ab = rsc_ab.drop(['Unnamed: 0'], axis = 1)
concepts

,Id,Name,LeafNode
3,4,Discrete,1
4,5,Realtime,1
6,7,Alternating,1
7,8,Simultaneous,1
8,9,Stochastic,1
9,10,HiddenInformation,1
10,11,Match,1
13,14,AsymmetricPlayRules,1
14,15,AsymmetricEndRules,1
16,17,AsymmetricSetup,1


Now that the df are filtered with the only attributes that we need, we will merge them in a final dataframe.

We will join using the following idea: 

Join Game Ruleset and Games on GameId and Id  --> Merge_One

Join Game Ruleset and Ruleset Concepts on Id and RulesetId --> Merge_two		

Join Concepts and Ruleset Concepts on Id and ConceptId --> Merge_three

In [42]:
def merge(one, two, on, left, right):
  new = pd.merge(one, two, on = on, left_on = left, right_on = right)

  return new

In [43]:
merge_one = merge(games, gamerules, None, 'Id', 'GameId')
merge_two = merge(rulesetconcept, gamerules, None, 'RulesetId', 'Id')
merge_three = merge(concepts, rulesetconcept, None, 'Id', 'ConceptId')

merge_three.drop(['Id_y'], axis=1, inplace=True)
merge_three = merge_three[merge_three['LeafNode']!=0]

Create a dataframe by merging the first three created above

In [44]:
df = merge(merge_one, merge_two, 'GameId', None, None)
df = df.drop([ 'Id_y_y', 'Id_x_x', 'Value', 'Id_x_y', 'ConceptId', 'Name_y', 'Id_y_x'], axis = 1);
df.head()

,Name_x,MainRuleset,LudiiRuleset,GameId,RulesetId,Name
0,Senet,528,529.0,3,529,Bell
1,Senet,528,529.0,3,529,Bell
2,Senet,528,529.0,3,529,Bell
3,Senet,528,529.0,3,529,Bell
4,Senet,528,529.0,3,529,Bell


Do the same merging as merge 3 but for the datasets of UCT and AB

In [45]:
merge_three_uct = pd.merge(concepts, rsc_uct, left_on='Id', right_on='ConceptId')

merge_three_uct.drop(['Id_y'], axis=1, inplace=True)
merge_three_uct = merge_three_uct[merge_three_uct['LeafNode']!=0]
merge_three_uct.head()

,Id_x,Name,LeafNode,RulesetId,ConceptId,Value
0,4,Discrete,1,1656,4,1.0
1,4,Discrete,1,1979,4,1.0
2,4,Discrete,1,35,4,1.0
3,4,Discrete,1,1391,4,1.0
4,4,Discrete,1,1796,4,1.0


In [46]:
merge_three_ab = pd.merge(concepts, rsc_ab, left_on='Id', right_on='ConceptId')

merge_three_ab.drop(['Id_y'], axis=1, inplace=True)
merge_three_ab = merge_three_ab[merge_three_ab['LeafNode']!=0]
merge_three_ab.head()

,Id_x,Name,LeafNode,RulesetId,ConceptId,Value
0,4,Discrete,1,1656,4,1.0
1,4,Discrete,1,1979,4,1.0
2,4,Discrete,1,35,4,1.0
3,4,Discrete,1,1391,4,1.0
4,4,Discrete,1,1796,4,1.0


In [47]:
def new_merge(merge_three):
  one = merge_three.groupby('RulesetId')['Name'].apply(list)
  one = one.to_frame()
  two = pd.merge(one, merge_three, on = 'RulesetId')

  three = two.groupby('RulesetId')['Value'].apply(list)
  three = three.to_frame()

  final = pd.merge(two, three, on = 'RulesetId')

  final.drop(['Name_y', 'LeafNode', 'ConceptId', 'Value_x', 'Id_x'], axis=1, inplace = True)
  final.drop_duplicates(subset = ['RulesetId'], inplace = True)
  return final

In [48]:
new_merge_ = new_merge(merge_three)

In [49]:
new_merge_uct = new_merge(merge_three_uct)

In [50]:
new_merge_ab = new_merge(merge_three_ab)

In [51]:
df = pd.merge(df, new_merge_, on = 'RulesetId')
df.rename(columns = {'Name_x_x' : 'Game', 'Name_x_y' : 'Concepts', 'Value_y' : 'Concepts_values'}, inplace = True)
df = pd.merge(df, new_merge_ab, on='RulesetId')
df.rename(columns = {'Name_x' : 'Concepts_AB', 'Value_y' : 'Concepts_values_AB'}, inplace = True)
df = pd.merge(df, new_merge_uct, on='RulesetId')
df.drop_duplicates(subset = ['RulesetId'], inplace = True)
df.rename(columns = {'Name_x' : 'Concepts_UCT', 'Value_y' : 'Concepts_values_UCT'}, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Concepts_AB,Concepts_values_AB,Concepts_UCT,Concepts_values_UCT
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


In [52]:
c_ab = df.iloc[0]['Concepts_AB']
for i in range(len(c_ab)):
  c_ab[i] = c_ab[i] + '_AB'

c_uct = df.iloc[0]['Concepts_UCT']
for i in range(len(c_uct)):
  c_uct[i] = c_uct[i] + '_UCT'

In [53]:
df[c_ab] = pd.DataFrame(df.Concepts_values_AB.tolist(), index= df.index)

df.drop(['Concepts_AB', 'Concepts_values_AB'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Concepts_UCT,Concepts_values_UCT,...,SetHiddenWhat_AB,SetHiddenWho_AB,CopyContext_AB,Then_AB,ForEachPiece_AB,DoLudeme_AB,Trigger_AB,PlayoutsPerSecond_AB,MovesPerSecond_AB,OutcomeUniformity_AB
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete_UCT, Realtime_UCT, Alternating_UCT, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1358.93,166173.49,NaN
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,155.01,107657.20,NaN
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,318.51,81394.19,NaN
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,321.79,81613.66,NaN
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,445.57,84795.32,NaN


In [54]:
df[c_uct] = pd.DataFrame(df.Concepts_values_UCT.tolist(), index= df.index)

df.drop(['Concepts_UCT', 'Concepts_values_UCT'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Concepts,Concepts_values,Discrete_AB,Realtime_AB,...,SetHiddenWhat_UCT,SetHiddenWho_UCT,CopyContext_UCT,Then_UCT,ForEachPiece_UCT,DoLudeme_UCT,Trigger_UCT,PlayoutsPerSecond_UCT,MovesPerSecond_UCT,OutcomeUniformity_UCT
0,Senet,528,529.0,3,529,Bell,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1246.27,152434.05,NaN
21784,Senet,528,529.0,3,939,Jequier,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,126.63,90267.74,NaN
43568,Senet,528,529.0,3,922,Kendall,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,259.06,65906.43,NaN
65352,Senet,528,529.0,3,937,Kendall All Options,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,273.09,66913.65,NaN
87136,Senet,528,529.0,3,923,Kendall Five Pieces,"[Discrete, Realtime, Alternating, Simultaneous...","[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0,0.0,...,0.0,0.0,1.0,1.0,1.0,1.0,0.0,396.11,75353.50,NaN


In [55]:
c=df['Concepts'].explode().unique()
df[c] = pd.DataFrame(df.Concepts_values.tolist(), index= df.index)

df.drop(['Concepts', 'Concepts_values'], axis=1, inplace = True)
df.head()

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Discrete_AB,Realtime_AB,Alternating_AB,Simultaneous_AB,...,SetHiddenValue,SetHiddenWhat,SetHiddenWho,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond
0,Senet,528,529.0,3,529,Bell,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1266.24,154613.78
21784,Senet,528,529.0,3,939,Jequier,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,337.43,87643.94
43568,Senet,528,529.0,3,922,Kendall,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,292.51,72076.99
65352,Senet,528,529.0,3,937,Kendall All Options,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,343.81,74169.23
87136,Senet,528,529.0,3,923,Kendall Five Pieces,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,445.21,84469.25


In [56]:
kilothonGames['RulesetId'] = kilothonGames['RulesetId'].astype('int')
df = pd.merge(df, kilothonGames, on = 'RulesetId')

In [57]:
df = df.T.drop_duplicates().T

### Now that the original data is prepared, we can start adding the heuristics data

In [310]:
heurs = pd.read_csv('heuristics.csv')
heurs = heurs.sort_values(by=['RulesetId', 'Value'], ascending=False)
heurs = heurs.drop(['Id'], axis=1)

In [311]:
selectedgames = pd.merge(df, abgames, on='Game', how='right')
total = pd.merge(selectedgames, heurs, on='RulesetId')
total

,Game,MainRuleset,LudiiRuleset,GameId,RulesetId,Name,Discrete_AB,Realtime_AB,Stochastic_AB,AsymmetricPiecesType_AB,...,Complement,ConditionalStatement,ControlFlowStatement,CopyContext,DoLudeme,PlayoutsPerSecond,MovesPerSecond,Name,HeuristicId,Value
0,Do Guti,918,918,787,918,Do Guti,1,0,0,0,...,0,0,1,0,0,17027.1,526666,Do Guti,13,61.842105
1,Do Guti,918,918,787,918,Do Guti,1,0,0,0,...,0,0,1,0,0,17027.1,526666,Do Guti,12,57.894737
2,Do Guti,918,918,787,918,Do Guti,1,0,0,0,...,0,0,1,0,0,17027.1,526666,Do Guti,6,55.500000
3,Do Guti,918,918,787,918,Do Guti,1,0,0,0,...,0,0,1,0,0,17027.1,526666,Do Guti,3,54.375000
4,Do Guti,918,918,787,918,Do Guti,1,0,0,0,...,0,0,1,0,0,17027.1,526666,Do Guti,8,51.770833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,Driesticken,952,952,802,952,Driesticken,1,0,0,0,...,0,0,1,0,0,22873.4,466905,Driesticken,6,45.882353
3912,Driesticken,952,952,802,952,Driesticken,1,0,0,0,...,0,0,1,0,0,22873.4,466905,Driesticken,12,45.789474
3913,Driesticken,952,952,802,952,Driesticken,1,0,0,0,...,0,0,1,0,0,22873.4,466905,Driesticken,2,45.238095
3914,Driesticken,952,952,802,952,Driesticken,1,0,0,0,...,0,0,1,0,0,22873.4,466905,Driesticken,14,44.523810


In [318]:

# random
endcols = pd.Series([
'LineWinFrequency',
'LineLossFrequency',
'ConnectionWinFrequency',
'ConnectionLossFrequency',
'GroupWinFrequency',
'LoopWinFrequency',
'TerritoryEndFrequency',
'CheckmateFrequency',
'CheckmateWinFrequency',
'NoTargetPieceWinFrequency',
'EliminatePiecesWinFrequency',
'EliminatePiecesDrawFrequency',
'NoOwnPiecesWinFrequency',
'NoOwnPiecesLossFrequency',
'FillEndFrequency',
'ReachWinFrequency',
'ScoringWinFrequency',
'ScoringLossFrequency',
'NoMovesWinFrequency',
'NoMovesLossFrequency',
'NoProgressLossFrequency',
'NoProgressDrawFrequency',
'DrawFrequency',])

# based on AB
endcols2 = pd.Series([
'LineWinFrequency_AB',
'LineLossFrequency_AB',
'ConnectionWinFrequency_AB',
'ConnectionLossFrequency_AB',
'GroupWinFrequency_AB',
'TerritoryEndFrequency_AB',
'CheckmateWinFrequency_AB',
'NoTargetPieceWinFrequency_AB',
'EliminatePiecesWinFrequency_AB',
'EliminatePiecesLossFrequency_AB',
'EliminatePiecesDrawFrequency_AB',
'NoOwnPiecesWinFrequency_AB',
'NoOwnPiecesLossFrequency_AB',
'FillWinFrequency_AB',
'ReachWinFrequency_AB',
'ReachLossFrequency_AB',
'ReachDrawFrequency_AB',
'ScoringWinFrequency_AB',
'ScoringLossFrequency_AB',
'NoMovesWinFrequency_AB',
'NoMovesLossFrequency_AB',
'NoMovesDrawFrequency_AB',
'NoProgressLossFrequency_AB',
'NoProgressDrawFrequency_AB',
'DrawFrequency_AB'
    
])



allgames = 0
finalheurs = []
for col in endcols2:
    subset = total[total[col]>0.1]
    if subset.RulesetId.unique().size !=0:
        allgames = allgames + subset.RulesetId.unique().size
        print(col, " ", subset.RulesetId.unique().size)
        thisend = subset.groupby('HeuristicId').mean().sort_values(by='Value', ascending=False)
        best = thisend['Value'].nlargest(3).index.tolist()
        finalheurs.append([col, subset.RulesetId.unique().size, best])
        print(thisend['Value'].nlargest(3).index.tolist())
        outputdf = pd.DataFrame([best])
        outputdf = outputdf.append(pd.DataFrame(subset['Game'].unique()))
#         outputdf.to_csv(col+'.csv') ONLY TURN ON IF YOU WANT ALL THE FILES
        pd.DataFrame(subset['Game'].unique()).to_csv('Games'+col+'.csv')
    
print('Total ends: ', allgames)
pd.DataFrame(finalheurs).to_csv('outputheuristics.csv')

onlyabgames.to_csv('ABGames2.csv')

LineWinFrequency_AB   7
[10, 17, 21]
LineLossFrequency_AB   1
[13, 11, 10]
GroupWinFrequency_AB   1
[16, 6, 19]
CheckmateWinFrequency_AB   24
[42, 9, 43]
NoTargetPieceWinFrequency_AB   1
[18, 20, 9]
EliminatePiecesWinFrequency_AB   2
[9, 1, 16]
NoOwnPiecesWinFrequency_AB   1
[7, 12, 13]
FillWinFrequency_AB   1
[23, 22, 31]
ReachLossFrequency_AB   1
[16, 17, 12]
ReachDrawFrequency_AB   1
[20, 1, 31]
ScoringWinFrequency_AB   2
[29, 17, 11]
NoMovesWinFrequency_AB   7
[9, 23, 22]
NoMovesLossFrequency_AB   1
[14, 20, 8]
NoProgressDrawFrequency_AB   3
[17, 64, 23]
DrawFrequency_AB   112
[64, 9, 43]
Total ends:  165


In [320]:
finalheurs

[['LineWinFrequency_AB', 7, [10, 17, 21]],
 ['LineLossFrequency_AB', 1, [13, 11, 10]],
 ['GroupWinFrequency_AB', 1, [16, 6, 19]],
 ['CheckmateWinFrequency_AB', 24, [42, 9, 43]],
 ['NoTargetPieceWinFrequency_AB', 1, [18, 20, 9]],
 ['EliminatePiecesWinFrequency_AB', 2, [9, 1, 16]],
 ['NoOwnPiecesWinFrequency_AB', 1, [7, 12, 13]],
 ['FillWinFrequency_AB', 1, [23, 22, 31]],
 ['ReachLossFrequency_AB', 1, [16, 17, 12]],
 ['ReachDrawFrequency_AB', 1, [20, 1, 31]],
 ['ScoringWinFrequency_AB', 2, [29, 17, 11]],
 ['NoMovesWinFrequency_AB', 7, [9, 23, 22]],
 ['NoMovesLossFrequency_AB', 1, [14, 20, 8]],
 ['NoProgressDrawFrequency_AB', 3, [17, 64, 23]],
 ['DrawFrequency_AB', 112, [64, 9, 43]]]